In [ ]:
import numpy as np
import pandas as pd
import random as rd

In [ ]:
datas_original = pd.read_csv("//content//french.txt")

In [ ]:
print(datas_original.head())

            AA
0     AALENIEN
1   AALENIENNE
2  AALENIENNES
3    AALENIENS
4          AAS


We will turn every words to the lower case

In [ ]:
datas_original["AA"] = datas_original["AA"].apply(lambda x: str(x).lower())


In [ ]:
datas = datas_original.copy()
datas = datas.rename(columns={'AA': 'a'})
print(datas.head())

             a
0     aalenien
1   aalenienne
2  aaleniennes
3    aaleniens
4          aas


General informations

In [ ]:
print("Nombre de datas: {}".format(len(datas)))

Nombre de datas: 95632


We remove words with less than 4 letters

In [ ]:
 #values va transformer en array
l = []
for i in datas["a"][:78000]:
  if len(str(i)) < 4:
    indice = list(datas["a"].values).index(i) #values va transformer en array
    l.append(indice)
print(l)
for i in l:
  datas = datas.drop(i)
print(len(datas))

[4, 3731, 5064, 5765, 8255, 8346, 8632, 9529, 9642, 9680, 9724, 9985, 10136, 10188, 10204, 10891, 12499, 13335, 13796, 14864, 14865, 15467, 16057, 16949, 18414, 20506, 20853, 21261, 22643, 22677, 22939, 22940, 23037, 26421, 28342, 28632, 29431, 29446, 29551, 29560, 29767, 29908, 30698, 30725, 31040, 31813, 32415, 33534, 33987, 34418, 34742, 34743, 34846, 35167, 35349, 35533, 35827, 36444, 36470, 36680, 37756, 38065, 38237, 38655, 39547, 40370, 40372, 40547, 40548, 40788, 40942, 41352, 41851, 44222, 44266, 47256, 47741, 47953, 48017, 48250, 48493, 48604, 48844, 48868, 48871, 49850, 50489, 53386, 54348, 56224, 57865, 58659, 59003, 59119, 65405, 65571, 65756, 65783, 66428, 67915, 69441, 70686, 70965, 72050, 74524]
95527


On ne recupere que les indices restants

In [ ]:
list_indices = datas.index[:]


In [ ]:
len(list_indices)

95527

In [ ]:
print(datas.head())
print(len(datas))

             a
0     aalenien
1   aalenienne
2  aaleniennes
3    aaleniens
5        abaca
95527


For each letter, we will determine the sequence in which they appear

Let us create a 26*26 table for it

In [ ]:
frequence_df = pd.DataFrame(np.zeros((26,26)), columns = [list("abcdefghijklmnopqrstuvwxyz")], index=list("abcdefghijklmnopqrstuvwxyz"))

In [ ]:
datas = datas.rename(columns={'a': 'mots'})

Function to determine the relation between letters

In [ ]:
def get_info_single_word(data, indice):
  mot =list(datas.loc[indice, "mots"])
  for i in range(len(mot)-1):
    pos1 = list("abcdefghijklmnopqrstuvwxyz").index(mot[i]) #Position de la lettre sur laquelle on se trouve
    pos2 = list("abcdefghijklmnopqrstuvwxyz").index(mot[i+1]) #Position de la lettre suivante
    frequence[pos1,pos2] = frequence[pos1,pos2] + 1

In [ ]:
print(list(datas.loc[3, "mots"]))
frequence = np.zeros((26,26))
for indice in list_indices[:77000]:#We test with only 77000 words
  get_info_single_word(datas, list_indices[indice])


['a', 'a', 'l', 'e', 'n', 'i', 'e', 'n', 's']


We save everything in a dataframe

In [ ]:
frequence_df = pd.DataFrame(frequence, columns=list("abcdefghijklmnopqrstuvwxyz"), index= list("abcdefghijklmnopqrstuvwxyz") )
print(frequence_df.head())

         a       b       c       d       e       f       g        h        i  \
a     15.0  3517.0  4863.0  2580.0   259.0  2198.0  3373.0    264.0  14526.0   
b   6139.0    27.0    40.0    56.0  2412.0     0.0     0.0     47.0   3709.0   
c  11234.0    37.0  1781.0    43.0  3683.0     2.0     0.0  10251.0   2886.0   
d   1772.0     4.0     0.0    99.0  2855.0     0.0   167.0     68.0   2512.0   
e   1228.0   290.0  1265.0   681.0  2972.0   217.0   749.0     73.0    531.0   

       j  ...      q        r        s       t       u       v     w      x  \
a  289.0  ...  708.0   8765.0  13788.0  8919.0  3603.0  1881.0  14.0  154.0   
b   48.0  ...    0.0   4411.0    287.0     2.0  1506.0     0.0   0.0    0.0   
c    0.0  ...  195.0    647.0     63.0  1238.0  1021.0     0.0   0.0    0.0   
d  242.0  ...    0.0    684.0    186.0     0.0   362.0    75.0   0.0    0.0   
e    7.0  ...  159.0  21906.0  16115.0  3836.0  2646.0   857.0   2.0  281.0   

       y       z  
a  334.0   308.0  
b   60

In [ ]:
print(frequence_df)

Pour chaque lettre, determinons celle la probabilite pour les autres de suivre:

We turn everything into probability

In [ ]:
frequence = frequence/frequence.sum()
for i in range(len(frequence[1][:])):
  frequence[i,:] = frequence[i,:]/frequence[i,:].sum()#On regroupe par ligne car c'est dans cet ordre qu'on a rempli
  #le dataset: celui qui suit est en colonne
print(frequence)

In [ ]:
print(frequence[13,19])

0.28618747309916354


Function to generate random letter

In [ ]:
def generate_random_letter(val,prob):
  pool = []
  for i in val:
    l = list("abcdefghijklmnopqrstuvwxyz")
    pool.append(l[i])

  random_letter = np.random.choice(pool, p=prob)
  return(random_letter)


Function to generate random word

In [ ]:
def main_part(first, length):
  mot = list(first)
  c = 0
  for i in range(1,length):

    #We take only the best possibilities
    indice = "abcdefghijklmnopqrstuvwxyz".index(mot[i-1])
    possibilities = sorted(frequence[indice,:])[23:26]# we store the index of the 5 letters with the highest probability
    high_prob_indexes = []
    for poss in possibilities:
      high_prob_indexes.append(np.where(frequence[indice,:] == poss)[0][0])
    s = sum(possibilities)
    for j in range(len(possibilities)):
      possibilities[j] = (possibilities[j]/s)# on ramene tt de facon a  avoir une somme qui fait 1

    letter = generate_random_letter(high_prob_indexes,possibilities)
    mot.append(letter)

    #We want to prevent 3 consonnes or vowels in a row:
    if (len(mot)) > 2:
      while mot[i] in "bcdfghjklmnpqrstvwxz" and (mot[i-1] in "bcdfghjklmnpqrstvwxz" and mot[i-2] in "bcdfghjklmnpqrstvwxz" ):
        mot[i] = generate_random_letter(high_prob_indexes,possibilities)
      while mot[i] in "aeiouy" and (mot[i-1] in "aeiouy" and mot[i-2] in "aeiouy"):
        mot[i] = generate_random_letter(high_prob_indexes, possibilities)

  my_string = ''.join(mot)#we create a word from the array
  return(my_string)

##Final program

In [ ]:
from pandas._libs.lib import map_infer_mask
n = int(input("How many words do you want?"))
dw= int(input("Give a range for the length of the words:\n(low value) "))
hi = int(input("Give a range for the length of the words:\n(high value)"))
first = input("Enter the first letter: ")
s = np.ones((hi-dw))/(hi-dw)
for i in range(n):
  length = np.random.choice(range(dw, hi), p = s)
  print(main_part(first, length))

How many words do you want?20
Give a range for the length of the words:
(low value) 4
Give a range for the length of the words:
(high value)8
Enter the first letter: do
doamiess
doinet
doenreen
doerraii
doines
doense
doannetn
doenn
doanne
doanses
doanses
doense
doens
doamnets
doinetsa
doanisos
doinetni
doersees
doimopup
doinses
